In [1]:
import pandas as pd
from datasets import load_dataset
from PIL import Image
import numpy as np
import time
import openai
import os

In [2]:

dataset = load_dataset('poloclub/diffusiondb', 'large_first_1k', trust_remote_code=True)
df=pd.DataFrame(dataset)
df.head()

,train
0,{'image': <PIL.WebPImagePlugin.WebPImageFile i...
1,{'image': <PIL.WebPImagePlugin.WebPImageFile i...
2,{'image': <PIL.WebPImagePlugin.WebPImageFile i...
3,{'image': <PIL.WebPImagePlugin.WebPImageFile i...
4,{'image': <PIL.WebPImagePlugin.WebPImageFile i...


In [41]:

dataset = load_dataset('poloclub/diffusiondb', 'large_first_50k', trust_remote_code=True)




In [43]:

batch_size = 10000
df_list = []  # List to store DataFrame batches

# Process dataset in batches
for i in range(0, len(dataset['train']), batch_size):
    batch = dataset['train'].select(range(i, min(i + batch_size, len(dataset['train']))))
    df_batch = batch.to_pandas()
    df_list.append(df_batch)  # Append batch to list

# Concatenate all batches into a final DataFrame
prompts = pd.concat(df_list, ignore_index=True)
secdeset = pd.read_csv('diffusion_db_unaltered.csv')
prompts = pd.concat([prompts, secdeset], ignore_index=True)

prompts.info()
# Display the first few rows
prompts.head()
prompts.tail()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2050000 entries, 0 to 2049999
Data columns (total 12 columns):
 #   Column       Dtype              
---  ------       -----              
 0   image        object             
 1   prompt       object             
 2   seed         float64            
 3   step         float64            
 4   cfg          float32            
 5   sampler      object             
 6   width        float64            
 7   height       float64            
 8   user_name    object             
 9   timestamp    datetime64[us, UTC]
 10  image_nsfw   float64            
 11  prompt_nsfw  float64            
dtypes: datetime64[us, UTC](1), float32(1), float64(6), object(4)
memory usage: 179.9+ MB


,image,prompt,seed,step,cfg,sampler,width,height,user_name,timestamp,image_nsfw,prompt_nsfw
2049995,NaN,david bowie giving a piggy back ride to ziggy ...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT,0.152736,0.022676
2049996,NaN,david bowie giving a piggy back ride to ziggy ...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT,0.127884,0.014115
2049997,NaN,funny computer,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT,0.093476,0.000463
2049998,NaN,hilarious witty computing machine,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT,0.124379,0.002547
2049999,NaN,"hilarious witty computing machine, lichtenstien",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT,0.044811,0.000964


In [5]:
image_path = prompts.loc[0, 'image']['path']  # Extract the image path
image = Image.open(image_path)
image.show()
image_array = np.array(image)
print(image_array.shape)

(512, 704, 3)


In [6]:
image_rgb = image.convert('RGB')
image_array = np.expand_dims(image_rgb, axis=0)


In [7]:
print(image_array)

[[[[130 154 160]
   [130 153 163]
   [128 154 164]
   ...
   [129 142 160]
   [129 143 163]
   [132 144 158]]

  [[127 152 163]
   [129 154 166]
   [128 153 165]
   ...
   [128 141 162]
   [127 140 163]
   [128 140 164]]

  [[127 152 166]
   [127 153 166]
   [128 155 167]
   ...
   [129 141 162]
   [130 142 163]
   [130 142 162]]

  ...

  [[115  99 120]
   [116  97 123]
   [117  99 125]
   ...
   [123  96 121]
   [124  98 123]
   [122  97 119]]

  [[115  99 121]
   [117  98 124]
   [116  96 123]
   ...
   [123  94 121]
   [122  97 120]
   [124 101 119]]

  [[112  98 116]
   [116  98 123]
   [115  98 124]
   ...
   [121  99 122]
   [118  96 119]
   [121 101 117]]]]


In [8]:
openai.api_key = os.getenv("OPENAI_API_KEY")
import getpass
if openai.api_key is None:
    print(' ValueError("API key is missing. Please set the OPENAI_API_KEY environment variable.")')
    openai.api_key = getpass.getpass("Enter your API key: ")
# Initialize the OpenAI client
client = openai

In [9]:
def classify_word(word):
    # Define a system message and user prompt to classify each word
    prompt = f"Classify the word '{word}' into one of the following categories: 'illegal', 'explicit', 'violent', 'innocent'. Provide just the category as the output."

    # Call the OpenAI API with the prompt
    try:
        response = client.chat.completions.create(
            model="gpt-4o-mini",
            messages=[
                {"role": "system", "content": "You are a helpful assistant that classifies words."},
                {"role": "user", "content": prompt}
            ],
            max_tokens=10
        )
        
        # Parse the response and extract the classification
        classification = response.choices[0].message.content.strip()
        if 'illegal' in classification:
            classification = 'illegal'
        elif 'explicit' in classification:
            classification = 'explicit'
        elif 'violent' in classification:
            classification = 'violent'
        elif 'innocent' in classification:
            classification = 'innocent'
        # Map the response to a unique value
        category_mapping = {
            "illegal": 1,
            "explicit": 2,
            "violent": 3,
            "innocent": 0
        }
        
        # Return the mapped value
        return category_mapping.get(classification, -1)  # Return -1 for unknown categories
    
    except Exception as e:
        print(f"Error classifying word '{word}': {e}")
        return -1

In [10]:
import re

def numberify(word):
    # Define the user prompt
    prompt = f"HERE IS THE WORD: {word}"

    try:
        # First classification: 4-digit number
        response = client.chat.completions.create(
            model="gpt-4o-mini",
            messages=[
                {"role": "system", "content": """Assign the word a number based on the following criteria.
    The number will be 4 digits. The first 4 digits determine how 'bad' the word may be.
    - Filler words = 1111
    - Non-explicit subjects = 2222
    - Non-explicit verbs = 3333
    - Explicit words increase progressively (e.g., 5555, 6666, 7777, 8888)
    - Use 9999 only for extreme cases.
    Respond with just the number!."""},
                {"role": "user", "content": prompt}
            ],
            max_tokens=10
        )
        classification = response.choices[0].message.content.strip()

        # Second classification: 2-digit number based on implication
        response = client.chat.completions.create(
            model="gpt-4o-mini",
            messages=[
                {"role": "system", "content": """Assign the word a number from 00 to 99 based on severity.
    - Use a balanced scale with 99 reserved for extreme words.
    Respond with just the number!."""},
                {"role": "user", "content": prompt}
            ],
            max_tokens=10
        )
        secondary = response.choices[0].message.content.strip()

        # Third classification: Letter count in 2-digit format
        letter_count = len(re.findall(r'[a-zA-Z]', word))
        formatted_count = str(letter_count).zfill(2)  # Ensures 2-digit format

        # Combine all values into a single 8-digit number
        final_number = f"{classification}{formatted_count}{secondary}"

        return final_number, word  # Returns the combined number and word
    
    except Exception as e:
        print(f"Error classifying word '{word}': {e}")
        return -1, word  # Returns -1 on error



In [11]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 1 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   train   1000 non-null   object
dtypes: object(1)
memory usage: 7.9+ KB


In [31]:
"""
import re
from tqdm import tqdm

# Example DataFrame (replace this with your actual DataFrame)

word_classifications = {}
rows = []
words_set = set()
# Loop through the DataFrame and classify words across all rows
for idx, row in tqdm(prompts.iterrows(), total=len(prompts), desc="Processing prompts"):
    prompt = row['prompt']
    for word in re.findall(r'\b\w+\b', prompt):  # Split the prompt into individual words
        if word not in words_set:  # Avoid classifying the same word twice
            words_set.add(word)
            word_classifications[word] = {}
            classification_value = classify_word(word)
            numeric_value, interpreted_word=numberify(word)
            rows.append([word, classification_value, numeric_value, interpreted_word])
            print(word, classification_value, numeric_value, interpreted_word)
        # Sleep to avoid hitting API rate limits
        time.sleep(.1)  # Adjust the sleep time as necessary

# Convert the dictionary to a DataFrame for better visualization
word_classification_df = pd.DataFrame(rows, columns=["Word", "Classification", "Number", "NumericInterpretedWord"])
# Display the final DataFrame with words and their classifications
print(word_classification_df)
"""

NameError: name 'tqdm' is not defined

In [52]:
import re
from tqdm import tqdm

def normalize_text(text):
    replacements = {
        '3': 'E',   # '3' as 'E'
        '5': 'S',   # '5' as 'S'
        '0': 'O',   # '0' as 'O'
        '1': 'I',   # '1' as 'I'
        '2': 'Z',   # '2' as 'Z'
        # Add more replacements as needed
    }
    for key, value in replacements.items():
        text = text.replace(key, value)
    return text
all_words = set()
prompts['prompt'] = prompts['prompt'].fillna("").astype(str)

for prompt in tqdm(prompts['prompt'], desc="Extracting unique words"):
    prompt = normalize_text(prompt)
    all_words.update(re.findall(r'\b\w+\b', prompt))
print(f"Total unique words: {len(all_words)}")
word_classifications = {}
rows = []
for word in tqdm(all_words, desc="Classifying words"):
    classification_value = classify_word(word)
    numeric_value, interpreted_word = numberify(word)
    word_classifications[word] = {
        'Classification': classification_value,
        'Number': numeric_value,
        'NumericInterpretedWord': interpreted_word
    }
    rows.append([word, classification_value, numeric_value, interpreted_word])
df = pd.DataFrame(rows, columns=["Word", "Classification", "Number", "NumericInterpretedWord"])

print(df)

Extracting unique words: 100%|██████████| 2050000/2050000 [00:28<00:00, 71369.57it/s]


Total unique words: 196211


Classifying words:   0%|          | 0/196211 [00:01<?, ?it/s]


KeyboardInterrupt: 

In [15]:
print(df['Classification'].unique())

[ 0  3  2  1 -1]


In [14]:
df = pd.DataFrame(rows, columns=["Word", "Classification", "Number", "NumericInterpretedWord"])


In [53]:
print(df.loc[df['Classification'] == -1])

Empty DataFrame
Columns: [Word, Classification, Number, NumericInterpretedWord]
Index: []


In [54]:
df.loc[df['Classification'] == -1, 'Classification'] = 0

In [97]:
len(df)

6664

In [96]:
count_class_1 = len(df.loc[df['Classification'] == 1])
count_class_2 = len(df.loc[df['Classification'] == 2])
count_class_3 = len(df.loc[df['Classification'] == 3])

# Print the results
print(f"Count of Classification 1: {count_class_1}")
print(f"Count of Classification 2: {count_class_2}")
print(f"Count of Classification 3: {count_class_3}")

Count of Classification 1: 64
Count of Classification 2: 53
Count of Classification 3: 199


In [55]:
print(df.loc[df['Classification'] == 2])

            Word  Classification    Number NumericInterpretedWord
37          evil               2  66660485                   evil
130     detailed               2  33330830               detailed
136       rapper               2  22220610                 rapper
391        scumm               2  88880585                  scumm
468        dirty               2  66660534                  dirty
604    seductive               2  33330930              seductive
973        devil               2  22220585                  devil
1070        sexy               2  55550445                   sexy
1157  suggestive               2  66661025             suggestive
1293       booty               2  66660515                  booty
1560        hell               2  55550485                   hell
1612        damn               2  55550425                   damn
1703  disturbing               2  66661065             disturbing
1792     command               2  33330745                command
1864    de

In [56]:
df.to_csv('WordList.csv', index=False)


PermissionError: [Errno 13] Permission denied: 'WordList.csv'

In [57]:
"""
Conditions: combination of 2 or more illicit classifications 1+2, 2+3, 1+3
Repetition: 3 or more repitions of the same illicit classification
average of numbers: if the average number is over 50000000
High number: If there is any instance of a number over 60000000
"""

'\nConditions: combination of 2 or more illicit classifications 1+2, 2+3, 1+3\nRepetition: 3 or more repitions of the same illicit classification\naverage of numbers: if the average number is over 50000000\nHigh number: If there is any instance of a number over 60000000\n'

In [58]:
SafePrompt='Generate an image of a child looking to the starry night'
NSFWPrompt='Generate an image of a busty looking female with big boobs'

In [59]:
prompts.head()

,image,prompt,seed,step,cfg,sampler,width,height,user_name,timestamp,image_nsfw,prompt_nsfw
0,"{'bytes': None, 'path': 'C:\Users\georg\.cache...","goddess portrait, ismail inceoglu",9.119600e+08,50.0,10.0,k_lms,704.0,512.0,d4393f4711a09b71abe9ac39a849b68d70f88ad15b94b0...,2022-08-20 09:32:00+00:00,0.055936,0.007284
1,"{'bytes': None, 'path': 'C:\Users\georg\.cache...","goddess portrait, ismail inceoglu",3.614154e+09,50.0,10.0,k_lms,704.0,512.0,d4393f4711a09b71abe9ac39a849b68d70f88ad15b94b0...,2022-08-20 09:32:00+00:00,0.103962,0.007284
2,"{'bytes': None, 'path': 'C:\Users\georg\.cache...",portrait of king of candy mr harry haribo oil ...,2.271109e+09,50.0,7.0,k_lms,512.0,512.0,7c375c633f3ac3961453837abde94580b043b1c2eb5abc...,2022-08-20 09:33:00+00:00,0.034399,0.022897
3,"{'bytes': None, 'path': 'C:\Users\georg\.cache...",super epic realistic nature photo trending on ...,1.947975e+09,50.0,7.0,k_lms,832.0,1024.0,bd227929e003e69eee392c1d8a409445dd790455c1e9bd...,2022-08-20 09:34:00+00:00,0.113144,0.001688
4,"{'bytes': None, 'path': 'C:\Users\georg\.cache...",super epic realistic nature photo trending on ...,1.035761e+09,50.0,7.0,k_lms,832.0,1024.0,bd227929e003e69eee392c1d8a409445dd790455c1e9bd...,2022-08-20 09:34:00+00:00,0.096025,0.001688


In [60]:
prompts['allowed'] = (prompts['prompt_nsfw'] < 0.5).astype(int)

In [61]:
prompts.head()

,image,prompt,seed,step,cfg,sampler,width,height,user_name,timestamp,image_nsfw,prompt_nsfw,allowed
0,"{'bytes': None, 'path': 'C:\Users\georg\.cache...","goddess portrait, ismail inceoglu",9.119600e+08,50.0,10.0,k_lms,704.0,512.0,d4393f4711a09b71abe9ac39a849b68d70f88ad15b94b0...,2022-08-20 09:32:00+00:00,0.055936,0.007284,1
1,"{'bytes': None, 'path': 'C:\Users\georg\.cache...","goddess portrait, ismail inceoglu",3.614154e+09,50.0,10.0,k_lms,704.0,512.0,d4393f4711a09b71abe9ac39a849b68d70f88ad15b94b0...,2022-08-20 09:32:00+00:00,0.103962,0.007284,1
2,"{'bytes': None, 'path': 'C:\Users\georg\.cache...",portrait of king of candy mr harry haribo oil ...,2.271109e+09,50.0,7.0,k_lms,512.0,512.0,7c375c633f3ac3961453837abde94580b043b1c2eb5abc...,2022-08-20 09:33:00+00:00,0.034399,0.022897,1
3,"{'bytes': None, 'path': 'C:\Users\georg\.cache...",super epic realistic nature photo trending on ...,1.947975e+09,50.0,7.0,k_lms,832.0,1024.0,bd227929e003e69eee392c1d8a409445dd790455c1e9bd...,2022-08-20 09:34:00+00:00,0.113144,0.001688,1
4,"{'bytes': None, 'path': 'C:\Users\georg\.cache...",super epic realistic nature photo trending on ...,1.035761e+09,50.0,7.0,k_lms,832.0,1024.0,bd227929e003e69eee392c1d8a409445dd790455c1e9bd...,2022-08-20 09:34:00+00:00,0.096025,0.001688,1


In [62]:
print(prompts['allowed'].unique())

[1 0]


In [63]:
prompts.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2050000 entries, 0 to 2049999
Data columns (total 13 columns):
 #   Column       Dtype              
---  ------       -----              
 0   image        object             
 1   prompt       object             
 2   seed         float64            
 3   step         float64            
 4   cfg          float32            
 5   sampler      object             
 6   width        float64            
 7   height       float64            
 8   user_name    object             
 9   timestamp    datetime64[us, UTC]
 10  image_nsfw   float64            
 11  prompt_nsfw  float64            
 12  allowed      int32              
dtypes: datetime64[us, UTC](1), float32(1), float64(6), int32(1), object(4)
memory usage: 187.7+ MB


In [88]:
import re
from tqdm import tqdm
for idx, row in tqdm(prompts.head(5000).iterrows(), total=len(prompts), desc="Processing prompts"):
    Classprompt = row['prompt']
    Numprompt = row['prompt']

    Classprompt = re.sub(r'[^a-zA-Z\s]', '', Classprompt)  
    Numprompt = re.sub(r'[^a-zA-Z\s]', '', Numprompt) 
    for word in re.findall(r'\b\w+\b', Classprompt):  
        if word not in df['Word'].values:
            classification_value = classify_word(word)
            numeric_value, interpreted_word = numberify(word)
        else:
            classification_value = df.loc[df['Word'] == word, 'Classification'].values[0]
            numeric_value = df.loc[df['Word'] == word, 'Number'].values[0]

        Classprompt = re.sub(r'\b' + re.escape(word) + r'\b', str(classification_value), Classprompt)
        Numprompt = re.sub(r'\b' + re.escape(word) + r'\b', str(numeric_value), Numprompt)

    prompts.at[idx, 'Classprompt'] = str(Classprompt) 
    prompts.at[idx, 'Numprompt'] = str(Numprompt) 


Processing prompts:   0%|          | 1250/2050000 [04:45<129:53:15,  4.38it/s] 


KeyboardInterrupt: 

In [89]:
prompts.head(1240)

,image,prompt,seed,step,cfg,sampler,width,height,user_name,timestamp,image_nsfw,prompt_nsfw,allowed,Classprompt,Numprompt,Class 3 appearance count,Class 2 appearance count,Class 1 appearance count,Class Sum count,Guess allowed,Numbers,Average Number
0,"{'bytes': None, 'path': 'C:\Users\georg\.cache\huggingface\datasets\downloads\extracted\8b8225f4c51599e99c7d8b471661b86c16f7f7b1d783cb2e9ec11312031ea1aa\f396d2d6-0668-46d6-833c-6a7da4c1038b.webp'}","goddess portrait, ismail inceoglu",9.119600e+08,50.0,10.0,k_lms,704.0,512.0,d4393f4711a09b71abe9ac39a849b68d70f88ad15b94b0f0b554571bee4e6741,2022-08-20 09:32:00+00:00,0.055936,0.007284,1,0 0 0 0,22220712 22220810 22220600 11110800,0.0,0.0,0.0,0.0,1.0,"[22220712, 22220810, 22220600, 11110800]",1.944323e+07
1,"{'bytes': None, 'path': 'C:\Users\georg\.cache\huggingface\datasets\downloads\extracted\8b8225f4c51599e99c7d8b471661b86c16f7f7b1d783cb2e9ec11312031ea1aa\941d339b-5481-4ee2-b92d-d8a8d2cbfc50.webp'}","goddess portrait, ismail inceoglu",3.614154e+09,50.0,10.0,k_lms,704.0,512.0,d4393f4711a09b71abe9ac39a849b68d70f88ad15b94b0f0b554571bee4e6741,2022-08-20 09:32:00+00:00,0.103962,0.007284,1,0 0 0 0,22220712 22220810 22220600 11110800,0.0,0.0,0.0,0.0,1.0,"[22220712, 22220810, 22220600, 11110800]",1.944323e+07
2,"{'bytes': None, 'path': 'C:\Users\georg\.cache\huggingface\datasets\downloads\extracted\8b8225f4c51599e99c7d8b471661b86c16f7f7b1d783cb2e9ec11312031ea1aa\411fd6fe-8b57-430a-a8f6-60d51a8085d1.webp'}",portrait of king of candy mr harry haribo oil painting bloody conquest tap [ e,2.271109e+09,50.0,7.0,k_lms,512.0,512.0,7c375c633f3ac3961453837abde94580b043b1c2eb5abce4d2c609650b08f08b,2022-08-20 09:33:00+00:00,0.034399,0.022897,1,0 0 0 0 0 0 0 0 0 0 3 3 0 0,22220810 11110200 22220410 11110200 22220505 11110210 33330515 22220610 22220320 22220810 66660670 22220858 33330310 11110100,2.0,0.0,0.0,2.0,0.0,"[22220810, 11110200, 22220410, 11110200, 22220505, 11110210, 33330515, 22220610, 22220320, 22220810, 66660670, 22220858, 33330310, 11110100]",2.380761e+07
3,"{'bytes': None, 'path': 'C:\Users\georg\.cache\huggingface\datasets\downloads\extracted\8b8225f4c51599e99c7d8b471661b86c16f7f7b1d783cb2e9ec11312031ea1aa\6f169011-9809-48a6-9b48-cad6cb07e37f.webp'}",super epic realistic nature photo trending on instagram with lonely person in yellow raincoat standing at a distance,1.947975e+09,50.0,7.0,k_lms,832.0,1024.0,bd227929e003e69eee392c1d8a409445dd790455c1e9bd86ae9e32f4341a874f,2022-08-20 09:34:00+00:00,0.113144,0.001688,1,0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0,22220550 22220470 22220940 22220640 22220510 22220835 33330210 22220910 11110400 22220635 22220650 11110200 22220610 22220805 33330820 11110200 11110100 22220825,0.0,0.0,0.0,0.0,1.0,"[22220550, 22220470, 22220940, 22220640, 22220510, 22220835, 33330210, 22220910, 11110400, 22220635, 22220650, 11110200, 22220610, 22220805, 33330820, 11110200, 11110100, 22220825]",2.098613e+07
4,"{'bytes': None, 'path': 'C:\Users\georg\.cache\huggingface\datasets\downloads\extracted\8b8225f4c51599e99c7d8b471661b86c16f7f7b1d783cb2e9ec11312031ea1aa\f88c16f5-5c8c-4427-b50e-2f155204eb11.webp'}",super epic realistic nature photo trending on instagram with lonely person in yellow raincoat standing at a distance,1.035761e+09,50.0,7.0,k_lms,832.0,1024.0,bd227929e003e69eee392c1d8a409445dd790455c1e9bd86ae9e32f4341a874f,2022-08-20 09:34:00+00:00,0.096025,0.001688,1,0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0,22220550 22220470 22220940 22220640 22220510 22220835 33330210 22220910 11110400 22220635 22220650 11110200 22220610 22220805 33330820 11110200 11110100 22220825,0.0,0.0,0.0,0.0,1.0,"[22220550, 22220470, 22220940, 22220640, 22220510, 22220835, 33330210, 22220910, 11110400, 22220635, 22220650, 11110200, 22220610, 22220805, 33330820, 11110200, 11110100, 22220825]",2.098613e+07
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1235,"{'bytes': None, 'path': 'C:\Users\georg\.cache\huggingface\datasets\downloads\extracted\79e9a2920e766a87e

In [90]:
#pre filtering
prompts['Class 3 appearance count'] = prompts['Classprompt'].str.count('3')
prompts['Class 2 appearance count'] = prompts['Classprompt'].str.count('2')
prompts['Class 1 appearance count'] = prompts['Classprompt'].str.count('1')

prompts['Class Sum count'] = (
    prompts['Class 3 appearance count'] +
    prompts['Class 2 appearance count'] +
    prompts['Class 1 appearance count']
)

prompts.loc[prompts['Class 3 appearance count'] > 4, 'Guess allowed'] = 0
prompts.loc[prompts['Class 2 appearance count'] > 4, 'Guess allowed'] = 0
prompts.loc[prompts['Class 1 appearance count'] > 4, 'Guess allowed'] = 0
prompts.loc[prompts['Class Sum count'] > 4, 'Guess allowed'] = 0

prompts['Numbers'] = prompts['Numprompt'].fillna("").astype(str).str.findall(r'\d+').apply(lambda x: list(map(int, x)) if x else [])

prompts['Average Number'] = prompts['Numbers'].apply(lambda x: sum(x)/len(x) if x else 0)
prompts.loc[prompts['Average Number'] > 50000000, 'Guess allowed'] = 0
prompts.loc[prompts['Average Number'] <= 50000000, 'Guess allowed'] = 1

prompts.loc[prompts['Numbers'].apply(lambda x: any(num > 60000000 for num in x)), 'Guess allowed'] = 0

"""
Conditions: combination of 2 or more illicit classifications 1+2, 2+3, 1+3
Repetition: 3 or more repitions of the same illicit classification
average of numbers: if the average number is over 50000000
High number: If there is any instance of a number over 60000000
"""
prompts.head()
#prompts.tail()

,image,prompt,seed,step,cfg,sampler,width,height,user_name,timestamp,image_nsfw,prompt_nsfw,allowed,Classprompt,Numprompt,Class 3 appearance count,Class 2 appearance count,Class 1 appearance count,Class Sum count,Guess allowed,Numbers,Average Number
0,"{'bytes': None, 'path': 'C:\Users\georg\.cache\huggingface\datasets\downloads\extracted\8b8225f4c51599e99c7d8b471661b86c16f7f7b1d783cb2e9ec11312031ea1aa\f396d2d6-0668-46d6-833c-6a7da4c1038b.webp'}","goddess portrait, ismail inceoglu",9.119600e+08,50.0,10.0,k_lms,704.0,512.0,d4393f4711a09b71abe9ac39a849b68d70f88ad15b94b0f0b554571bee4e6741,2022-08-20 09:32:00+00:00,0.055936,0.007284,1,0 0 0 0,22220712 22220810 22220600 11110800,0.0,0.0,0.0,0.0,1.0,"[22220712, 22220810, 22220600, 11110800]",1.944323e+07
1,"{'bytes': None, 'path': 'C:\Users\georg\.cache\huggingface\datasets\downloads\extracted\8b8225f4c51599e99c7d8b471661b86c16f7f7b1d783cb2e9ec11312031ea1aa\941d339b-5481-4ee2-b92d-d8a8d2cbfc50.webp'}","goddess portrait, ismail inceoglu",3.614154e+09,50.0,10.0,k_lms,704.0,512.0,d4393f4711a09b71abe9ac39a849b68d70f88ad15b94b0f0b554571bee4e6741,2022-08-20 09:32:00+00:00,0.103962,0.007284,1,0 0 0 0,22220712 22220810 22220600 11110800,0.0,0.0,0.0,0.0,1.0,"[22220712, 22220810, 22220600, 11110800]",1.944323e+07
2,"{'bytes': None, 'path': 'C:\Users\georg\.cache\huggingface\datasets\downloads\extracted\8b8225f4c51599e99c7d8b471661b86c16f7f7b1d783cb2e9ec11312031ea1aa\411fd6fe-8b57-430a-a8f6-60d51a8085d1.webp'}",portrait of king of candy mr harry haribo oil painting bloody conquest tap [ e,2.271109e+09,50.0,7.0,k_lms,512.0,512.0,7c375c633f3ac3961453837abde94580b043b1c2eb5abce4d2c609650b08f08b,2022-08-20 09:33:00+00:00,0.034399,0.022897,1,0 0 0 0 0 0 0 0 0 0 3 3 0 0,22220810 11110200 22220410 11110200 22220505 11110210 33330515 22220610 22220320 22220810 66660670 22220858 33330310 11110100,2.0,0.0,0.0,2.0,0.0,"[22220810, 11110200, 22220410, 11110200, 22220505, 11110210, 33330515, 22220610, 22220320, 22220810, 66660670, 22220858, 33330310, 11110100]",2.380761e+07
3,"{'bytes': None, 'path': 'C:\Users\georg\.cache\huggingface\datasets\downloads\extracted\8b8225f4c51599e99c7d8b471661b86c16f7f7b1d783cb2e9ec11312031ea1aa\6f169011-9809-48a6-9b48-cad6cb07e37f.webp'}",super epic realistic nature photo trending on instagram with lonely person in yellow raincoat standing at a distance,1.947975e+09,50.0,7.0,k_lms,832.0,1024.0,bd227929e003e69eee392c1d8a409445dd790455c1e9bd86ae9e32f4341a874f,2022-08-20 09:34:00+00:00,0.113144,0.001688,1,0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0,22220550 22220470 22220940 22220640 22220510 22220835 33330210 22220910 11110400 22220635 22220650 11110200 22220610 22220805 33330820 11110200 11110100 22220825,0.0,0.0,0.0,0.0,1.0,"[22220550, 22220470, 22220940, 22220640, 22220510, 22220835, 33330210, 22220910, 11110400, 22220635, 22220650, 11110200, 22220610, 22220805, 33330820, 11110200, 11110100, 22220825]",2.098613e+07
4,"{'bytes': None, 'path': 'C:\Users\georg\.cache\huggingface\datasets\downloads\extracted\8b8225f4c51599e99c7d8b471661b86c16f7f7b1d783cb2e9ec11312031ea1aa\f88c16f5-5c8c-4427-b50e-2f155204eb11.webp'}",super epic realistic nature photo trending on instagram with lonely person in yellow raincoat standing at a distance,1.035761e+09,50.0,7.0,k_lms,832.0,1024.0,bd227929e003e69eee392c1d8a409445dd790455c1e9bd86ae9e32f4341a874f,2022-08-20 09:34:00+00:00,0.096025,0.001688,1,0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0,22220550 22220470 22220940 22220640 22220510 22220835 33330210 22220910 11110400 22220635 22220650 11110200 22220610 22220805 33330820 11110200 11110100 22220825,0.0,0.0,0.0,0.0,1.0,"[22220550, 22220470, 22220940, 22220640, 22220510, 22220835, 33330210, 22220910, 11110400, 22220635, 22220650, 11110200, 22220610, 22220805, 33330820, 11110200, 11110100, 22220825]",2.098613e+07


In [91]:

pd.set_option('display.max_columns', None)  # Show all columns
pd.set_option('display.max_colwidth', None)
print(prompts.iloc[2])

image                       {'bytes': None, 'path': 'C:\Users\georg\.cache\huggingface\datasets\downloads\extracted\8b8225f4c51599e99c7d8b471661b86c16f7f7b1d783cb2e9ec11312031ea1aa\411fd6fe-8b57-430a-a8f6-60d51a8085d1.webp'}
prompt                                                                                                                                           portrait of king of candy mr harry haribo oil painting bloody conquest tap [ e 
seed                                                                                                                                                                                                                2271108630.0
step                                                                                                                                                                                                                        50.0
cfg                                                                                                 

In [104]:
df_subset = prompts.head(1240)
if 'Guess allowed' in df_subset.columns and 'allowed' in df_subset.columns:
    true_positives = (df_subset['Guess allowed'] == 1) & (df_subset['allowed'] == 1)

    true_negatives = (df_subset['Guess allowed'] == 0) & (df_subset['allowed'] == 0)

    false_positives = (df_subset['Guess allowed'] == 1) & (df_subset['allowed'] == 0)

    false_negatives = (df_subset['Guess allowed'] == 0) & (df_subset['allowed'] == 1)

    tp_count = true_positives.sum()
    tn_count = true_negatives.sum()
    fp_count = false_positives.sum()
    fn_count = false_negatives.sum()

    print(f"True Positives (TP): {tp_count}")
    print(f"True Negatives (TN): {tn_count}")
    print(f"False Positives (FP): {fp_count}")
    print(f"False Negatives (FN): {fn_count}")
    total = len(df_subset)

    tp_percentage = (tp_count / total) * 100
    tn_percentage = (tn_count / total) * 100
    fp_percentage = (fp_count / total) * 100
    fn_percentage = (fn_count / total) * 100

    print(f"True Positives Ratio: {tp_percentage:.2f}%")
    print(f"True Negatives Ratio: {tn_percentage:.2f}%")
    print(f"False Positives Ratio: {fp_percentage:.2f}%")
    print(f"False Negatives Ratio: {fn_percentage:.2f}%")

    accuracy = (tp_count + tn_count) / len(df_subset)
    print(f"Accuracy: {accuracy * 100:.2f}%")
else:
    print("The required columns are not present in the first 100 rows.")

True Positives (TP): 1146
True Negatives (TN): 4
False Positives (FP): 13
False Negatives (FN): 77
True Positives Ratio: 92.42%
True Negatives Ratio: 0.32%
False Positives Ratio: 1.05%
False Negatives Ratio: 6.21%
Accuracy: 92.74%


In [93]:
import re
from tqdm import tqdm

# Define your custom prompt here (replace this with your actual prompt)
custom_prompt = NSFWPrompt

# Initialize the word classifications (use your pre-built logic)
word_classifications = {}
rows = []

# Process the custom prompt
Classprompt = custom_prompt
Numprompt = custom_prompt

# Clean up non-alphabetical characters (remove special characters)
Classprompt = re.sub(r'[^a-zA-Z\s]', '', Classprompt)  
Numprompt = re.sub(r'[^a-zA-Z\s]', '', Numprompt)

# Process words and classify them
for word in re.findall(r'\b\w+\b', Classprompt):  # Loop through each word
    if word not in word_classifications:
        classification_value = classify_word(word)  # Classify word (you'll have to define this)
        numeric_value, interpreted_word = numberify(word)  # Interpret numeric value (define this too)
        word_classifications[word] = {'Classification': classification_value, 
                                       'Number': numeric_value, 'NumericInterpretedWord': interpreted_word}
    else:
        classification_value = word_classifications[word]['Classification']
        numeric_value = word_classifications[word]['Number']
    
    # Replace the word in the prompts with the classification and numeric value
    Classprompt = re.sub(r'\b' + re.escape(word) + r'\b', str(classification_value), Classprompt)
    Numprompt = re.sub(r'\b' + re.escape(word) + r'\b', str(numeric_value), Numprompt)

# Output the processed prompt with classifications and numeric values
print("Processed Classprompt:", Classprompt)
print("Processed Numprompt:", Numprompt)

# Pre-filtering based on the same logic as before
Class_3_count = Classprompt.count('3')
Class_2_count = Classprompt.count('2')
Class_1_count = Classprompt.count('1')

Class_Sum_count = Class_3_count + Class_2_count + Class_1_count

# Apply filtering conditions
Guess_allowed = 1  # Default assumption (if conditions are not violated)
if Class_3_count > 4 or Class_2_count > 4 or Class_1_count > 4 or Class_Sum_count > 4:
    Guess_allowed = 0

# Extract numbers from Numprompt
Numbers = list(map(int, re.findall(r'\d+', Numprompt)))
Average_Number = sum(Numbers)/len(Numbers) if Numbers else 0

if Average_Number > 50000000:
    Guess_allowed = 0
elif any(num > 60000000 for num in Numbers):
    Guess_allowed = 0

# Print the result for the custom prompt
print(f"Guess allowed: {Guess_allowed}")

# Assuming you have the actual label `allowed` for your custom prompt (manually define it here)
# For this example, let's assume the actual label is stored in `actual_allowed`:
actual_allowed = 0  # You would replace this with the actual expected result for your custom prompt

# Calculate the accuracy or compare the results
if Guess_allowed == actual_allowed:
    print("Prediction is correct.")
else:
    print("Prediction is incorrect.")


Processed Classprompt: 0 0 0 0 0 2 0 0 0 0 2
Processed Numprompt: 33330840 11110200 22220530 11110200 11110100 66660535 33330710 22220610 11110400 33330325 66660520
Guess allowed: 0
Prediction is correct.


In [94]:
import re
from tqdm import tqdm

# Define your custom prompt here (replace this with your actual prompt)
custom_prompt = SafePrompt

# Initialize the word classifications (use your pre-built logic)
word_classifications = {}
rows = []

# Process the custom prompt
Classprompt = custom_prompt
Numprompt = custom_prompt

# Clean up non-alphabetical characters (remove special characters)
Classprompt = re.sub(r'[^a-zA-Z\s]', '', Classprompt)  
Numprompt = re.sub(r'[^a-zA-Z\s]', '', Numprompt)

# Process words and classify them
for word in re.findall(r'\b\w+\b', Classprompt):  # Loop through each word
    if word not in word_classifications:
        classification_value = classify_word(word)  # Classify word (you'll have to define this)
        numeric_value, interpreted_word = numberify(word)  # Interpret numeric value (define this too)
        word_classifications[word] = {'Classification': classification_value, 
                                       'Number': numeric_value, 'NumericInterpretedWord': interpreted_word}
    else:
        classification_value = word_classifications[word]['Classification']
        numeric_value = word_classifications[word]['Number']
    
    # Replace the word in the prompts with the classification and numeric value
    Classprompt = re.sub(r'\b' + re.escape(word) + r'\b', str(classification_value), Classprompt)
    Numprompt = re.sub(r'\b' + re.escape(word) + r'\b', str(numeric_value), Numprompt)

# Output the processed prompt with classifications and numeric values
print("Processed Classprompt:", Classprompt)
print("Processed Numprompt:", Numprompt)

# Pre-filtering based on the same logic as before
Class_3_count = Classprompt.count('3')
Class_2_count = Classprompt.count('2')
Class_1_count = Classprompt.count('1')

Class_Sum_count = Class_3_count + Class_2_count + Class_1_count

# Apply filtering conditions
Guess_allowed = 1  # Default assumption (if conditions are not violated)
if Class_3_count > 4 or Class_2_count > 4 or Class_1_count > 4 or Class_Sum_count > 4:
    Guess_allowed = 0

# Extract numbers from Numprompt
Numbers = list(map(int, re.findall(r'\d+', Numprompt)))
Average_Number = sum(Numbers)/len(Numbers) if Numbers else 0

if Average_Number > 50000000:
    Guess_allowed = 0
elif any(num > 60000000 for num in Numbers):
    Guess_allowed = 0

# Print the result for the custom prompt
print(f"Guess allowed: {Guess_allowed}")

# Assuming you have the actual label `allowed` for your custom prompt (manually define it here)
# For this example, let's assume the actual label is stored in `actual_allowed`:
actual_allowed = 1  # You would replace this with the actual expected result for your custom prompt

# Calculate the accuracy or compare the results
if Guess_allowed == actual_allowed:
    print("Prediction is correct.")
else:
    print("Prediction is incorrect.")


Processed Classprompt: 0 0 0 0 0 0 0 0 0 0 0
Processed Numprompt: 33330825 11110200 22220535 11110200 11110100 22220510 33330710 11110200 11110300 22220610 22220510
Guess allowed: 1
Prediction is correct.


In [98]:
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score

In [106]:
X = df_subset['Numbers'].apply(lambda x: sum(x) if isinstance(x, list) else 0).values.reshape(-1, 1)  # If 'Numbers' is a list, sum them as a feature
y = df_subset['allowed']

# Split the data into train and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

# Loop over different values of k and compute the accuracy
for k in range(1, 13):  # Test values of k from 1 to 12
    print("="*50)
    print(f"Results for k={k}")
    print("="*50)
    
    knn = KNeighborsClassifier(n_neighbors=k)
    knn.fit(X_train, y_train)
    
    # Step 5: Make Predictions
    y_pred = knn.predict(X_test)
    
    # Step 6: Evaluate the Model
    accuracy = accuracy_score(y_test, y_pred)
    print(f'Accuracy for k={k}: {accuracy * 100:.2f}%')
    
    # Confusion Matrix
    cm = confusion_matrix(y_test, y_pred)
    print(f"\nConfusion Matrix for k={k}:")
    print(cm)
    
    # Extract the components of the confusion matrix
    tn, fp, fn, tp = cm.ravel()
    
    print(f"\nTrue Positives (TP): {tp}")
    print(f"True Negatives (TN): {tn}")
    print(f"False Positives (FP): {fp}")
    print(f"False Negatives (FN): {fn}")
    
    # Additional metrics (optional)
    precision = tp / (tp + fp) if tp + fp > 0 else 0
    recall = tp / (tp + fn) if tp + fn > 0 else 0
    f1_score = 2 * (precision * recall) / (precision + recall) if precision + recall > 0 else 0
    
    print(f"Precision for k={k}: {precision * 100:.2f}%")
    print(f"Recall for k={k}: {recall * 100:.2f}%")
    print(f"F1 Score for k={k}: {f1_score * 100:.2f}%")
    
    # Print the first 5 predicted values and their actual counterparts
    print("\nFirst 5 predictions vs actual values:")
    for i in range(5):
        print(f"Sample {i+1} - Predicted: {y_pred[i]}, Actual: {y_test.iloc[i]}")
    
    print("\n" + "="*50 + "\n")

Results for k=1
Accuracy for k=1: 100.00%

Confusion Matrix for k=1:
[[  3   0]
 [  0 369]]

True Positives (TP): 369
True Negatives (TN): 3
False Positives (FP): 0
False Negatives (FN): 0
Precision for k=1: 100.00%
Recall for k=1: 100.00%
F1 Score for k=1: 100.00%

First 5 predictions vs actual values:
Sample 1 - Predicted: 1, Actual: 1
Sample 2 - Predicted: 1, Actual: 1
Sample 3 - Predicted: 1, Actual: 1
Sample 4 - Predicted: 1, Actual: 1
Sample 5 - Predicted: 1, Actual: 1


Results for k=2
Accuracy for k=2: 100.00%

Confusion Matrix for k=2:
[[  3   0]
 [  0 369]]

True Positives (TP): 369
True Negatives (TN): 3
False Positives (FP): 0
False Negatives (FN): 0
Precision for k=2: 100.00%
Recall for k=2: 100.00%
F1 Score for k=2: 100.00%

First 5 predictions vs actual values:
Sample 1 - Predicted: 1, Actual: 1
Sample 2 - Predicted: 1, Actual: 1
Sample 3 - Predicted: 1, Actual: 1
Sample 4 - Predicted: 1, Actual: 1
Sample 5 - Predicted: 1, Actual: 1


Results for k=3
Accuracy for k=3: 10

In [111]:
import re
import numpy as np
from tqdm import tqdm

# Assuming the model 'knn' is already trained

# Define your custom prompt here (replace this with your actual prompt)
custom_prompt = NSFWPrompt

# Initialize the word classifications (use your pre-built logic)
word_classifications = {}
rows = []

# Process the custom prompt
Classprompt = custom_prompt
Numprompt = custom_prompt

# Clean up non-alphabetical characters (remove special characters)
Classprompt = re.sub(r'[^a-zA-Z\s]', '', Classprompt)  
Numprompt = re.sub(r'[^a-zA-Z\s]', '', Numprompt)

# Process words and classify them
for word in re.findall(r'\b\w+\b', Classprompt):  # Loop through each word
    if word not in word_classifications:
        # Here, classify the word based on your own logic, e.g., classify_word, numberify
        classification_value = classify_word(word)  # Classify word (you'll have to define this)
        numeric_value, interpreted_word = numberify(word)  # Interpret numeric value (define this too)
        word_classifications[word] = {'Classification': classification_value, 
                                       'Number': numeric_value, 'NumericInterpretedWord': interpreted_word}
    else:
        classification_value = word_classifications[word]['Classification']
        numeric_value = word_classifications[word]['Number']
    
    # Replace the word in the prompts with the classification and numeric value
    Classprompt = re.sub(r'\b' + re.escape(word) + r'\b', str(classification_value), Classprompt)
    Numprompt = re.sub(r'\b' + re.escape(word) + r'\b', str(numeric_value), Numprompt)

# Output the processed prompt with classifications and numeric values
print("Processed Classprompt:", Classprompt)
print("Processed Numprompt:", Numprompt)

# Pre-filtering based on the same logic as before
Class_3_count = Classprompt.count('3')
Class_2_count = Classprompt.count('2')
Class_1_count = Classprompt.count('1')

Class_Sum_count = Class_3_count + Class_2_count + Class_1_count

# Apply filtering conditions
Guess_allowed = 1  # Default assumption (if conditions are not violated)
if Class_3_count > 4 or Class_2_count > 4 or Class_1_count > 4 or Class_Sum_count > 4:
    Guess_allowed = 0

# Extract numbers from Numprompt
Numbers = list(map(int, re.findall(r'\d+', Numprompt)))
Average_Number = sum(Numbers)/len(Numbers) if Numbers else 0

if Average_Number > 50000000:
    Guess_allowed = 0
elif any(num > 60000000 for num in Numbers):
    Guess_allowed = 0

# Print the result for the custom prompt
print(f"Guess allowed: {Guess_allowed}")

# Assuming you have the actual label `allowed` for your custom prompt (manually define it here)
# For this example, let's assume the actual label is stored in `actual_allowed`:
actual_allowed = 0  # You would replace this with the actual expected result for your custom prompt

# Calculate the accuracy or compare the results
if Guess_allowed == actual_allowed:
    print("Prediction is correct.")
else:
    print("Prediction is incorrect.")

# Preprocess the custom prompt (as you did with your training data)
def preprocess_custom_prompt(prompt):
    # Extract numbers from the prompt
    numbers = re.findall(r'\d+', prompt)
    
    # Convert the numbers to integers
    numbers = list(map(int, numbers))
    
    # Sum the numbers (or any other feature engineering you want to use)
    sum_of_numbers = sum(numbers) if numbers else 0
    
    # Return the feature as an array (reshape to match the model input)
    return np.array([[sum_of_numbers]])

# Preprocess the custom prompt for KNN input
X_custom = preprocess_custom_prompt(custom_prompt)

# Predict the class for the custom prompt using KNN
knn = KNeighborsClassifier(n_neighbors=1)
knn.fit(X_train, y_train)
custom_prediction = knn.predict(X_custom)
if custom_prediction == actual_allowed:
    print("KNN Prediction is correct.")
else:
    print("KNN Prediction is incorrect.")
# Output the KNN prediction result
print(f"KNN Prediction for the custom prompt: {custom_prediction[0]}")


Error classifying word 'Generate': Connection error.
Error classifying word 'Generate': Connection error.
Error classifying word 'an': Connection error.
Error classifying word 'an': Connection error.
Error classifying word 'image': Connection error.
Error classifying word 'image': Connection error.
Error classifying word 'of': Connection error.
Error classifying word 'of': Connection error.
Error classifying word 'a': Connection error.
Error classifying word 'a': Connection error.
Error classifying word 'busty': Connection error.
Error classifying word 'busty': Connection error.
Error classifying word 'looking': Connection error.
Error classifying word 'looking': Connection error.


KeyboardInterrupt: 

In [112]:
import re
import numpy as np
from tqdm import tqdm

# Assuming the model 'knn' is already trained

# Define your custom prompt here (replace this with your actual prompt)
custom_prompt = SafePrompt

# Initialize the word classifications (use your pre-built logic)
word_classifications = {}
rows = []

# Process the custom prompt
Classprompt = custom_prompt
Numprompt = custom_prompt

# Clean up non-alphabetical characters (remove special characters)
Classprompt = re.sub(r'[^a-zA-Z\s]', '', Classprompt)  
Numprompt = re.sub(r'[^a-zA-Z\s]', '', Numprompt)

# Process words and classify them
for word in re.findall(r'\b\w+\b', Classprompt):  # Loop through each word
    if word not in word_classifications:
        # Here, classify the word based on your own logic, e.g., classify_word, numberify
        classification_value = classify_word(word)  # Classify word (you'll have to define this)
        numeric_value, interpreted_word = numberify(word)  # Interpret numeric value (define this too)
        word_classifications[word] = {'Classification': classification_value, 
                                       'Number': numeric_value, 'NumericInterpretedWord': interpreted_word}
    else:
        classification_value = word_classifications[word]['Classification']
        numeric_value = word_classifications[word]['Number']
    
    # Replace the word in the prompts with the classification and numeric value
    Classprompt = re.sub(r'\b' + re.escape(word) + r'\b', str(classification_value), Classprompt)
    Numprompt = re.sub(r'\b' + re.escape(word) + r'\b', str(numeric_value), Numprompt)

# Output the processed prompt with classifications and numeric values
print("Processed Classprompt:", Classprompt)
print("Processed Numprompt:", Numprompt)

# Pre-filtering based on the same logic as before
Class_3_count = Classprompt.count('3')
Class_2_count = Classprompt.count('2')
Class_1_count = Classprompt.count('1')

Class_Sum_count = Class_3_count + Class_2_count + Class_1_count

# Apply filtering conditions
Guess_allowed = 1  # Default assumption (if conditions are not violated)
if Class_3_count > 4 or Class_2_count > 4 or Class_1_count > 4 or Class_Sum_count > 4:
    Guess_allowed = 0

# Extract numbers from Numprompt
Numbers = list(map(int, re.findall(r'\d+', Numprompt)))
Average_Number = sum(Numbers)/len(Numbers) if Numbers else 0

if Average_Number > 50000000:
    Guess_allowed = 0
elif any(num > 60000000 for num in Numbers):
    Guess_allowed = 0

# Print the result for the custom prompt
print(f"Guess allowed: {Guess_allowed}")

# Assuming you have the actual label `allowed` for your custom prompt (manually define it here)
# For this example, let's assume the actual label is stored in `actual_allowed`:
actual_allowed = 0  # You would replace this with the actual expected result for your custom prompt

# Calculate the accuracy or compare the results
if Guess_allowed == actual_allowed:
    print("Prediction is correct.")
else:
    print("Prediction is incorrect.")

# Preprocess the custom prompt (as you did with your training data)
def preprocess_custom_prompt(prompt):
    # Extract numbers from the prompt
    numbers = re.findall(r'\d+', prompt)
    
    # Convert the numbers to integers
    numbers = list(map(int, numbers))
    
    # Sum the numbers (or any other feature engineering you want to use)
    sum_of_numbers = sum(numbers) if numbers else 0
    
    # Return the feature as an array (reshape to match the model input)
    return np.array([[sum_of_numbers]])

# Preprocess the custom prompt for KNN input
X_custom = preprocess_custom_prompt(custom_prompt)

# Predict the class for the custom prompt using KNN
knn = KNeighborsClassifier(n_neighbors=1)
knn.fit(X_train, y_train)
custom_prediction = knn.predict(X_custom)
if custom_prediction == actual_allowed:
    print("KNN Prediction is correct.")
else:
    print("KNN Prediction is incorrect.")
# Output the KNN prediction result
print(f"KNN Prediction for the custom prompt: {custom_prediction[0]}")


Error classifying word 'Generate': Connection error.
Error classifying word 'Generate': Connection error.
Error classifying word 'an': Connection error.
Error classifying word 'an': Connection error.


KeyboardInterrupt: 